In [1]:
import os
import pandas as pd
import geopandas as gpd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [2]:
data_path = os.path.join('..', 'data')
filename = 'Sustainability Literature review.csv'
data = os.path.join(data_path, filename)

df = pd.read_csv(data)

In [3]:
df.head(3)

,year,author_city,country,continent,number,income,publisher,meta_publisher,access,technology,sub_category,open_code,open_data,spatial_focus,SDG,Methodology,Application,Title
0,2022,Trondheim,Norway,Europe,1,NaN,IEEE,Access,Free,All,NaN,Not Applicable,Not Applicable,Not Applicable,All,Literature Review,General Information,A Tertiary Review on Blockchain and Sustainabi...
1,2018,Santiago,Chile,South America,1,NaN,IEEE,Access,Free,All,NaN,Not Applicable,Not Applicable,Not Applicable,All,Literature Review,General Information,Information and Communications Technologies fo...
2,2024,Pretoria,South Africa,North America,1,UMC,IEEE,Access,Free,All,NaN,Not Applicable,Not Applicable,Not Applicable,All,Experiment & Survey,General Information,Supporting the Measurement of Sustainable Deve...


Initialize Nominatim API

In [4]:
geolocator = Nominatim(user_agent = "bonfaceosoro@gmail.com")

Rate limit to avoid service overload

In [5]:
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

Function to get latitude and longitude

In [6]:
def get_coordinates(city):
    
    location = geolocator.geocode(city)
    
    if location:
        
        return pd.Series([location.latitude, location.longitude])
        
    else:
        
        return pd.Series([None, None])

In [7]:
df[['latitude', 'longitude']] = df['author_city'].apply(get_coordinates)

In [8]:
fileout = 'literature_coordinates.csv'
folder_out = os.path.join('..', 'results')

if not os.path.exists(folder_out):

    os.makedirs(folder_out)

path_out = os.path.join(folder_out, fileout)

In [9]:
gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.latitude, df.longitude))
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

/var/folders/26/c8kdf2_x3x5bcbqcrtq4hml80000gn/T/ipykernel_3907/2659378485.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [10]:
df.to_csv(path_out)